In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np

df = pd.read_csv("logistic regression dataset-Social_Network_Ads.csv")
print(df.info())
print(df.describe())
df.drop(columns=["User ID"], inplace=True)

df["Gender"] = df["Gender"].apply(lambda x: 1 if x == "Male" else 0)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB
None
            User ID         Age  EstimatedSalary   Purchased
count  4.000000e+02  400.000000       400.000000  400.000000
mean   1.569154e+07   37.655000     69742.500000    0.357500
std    7.165832e+04   10.482877     34096.960282    0.479864
min    1.556669e+07   18.000000     15000.000000    0.000000
25%    1.562676e+07   29.750000     43000.000000    0.000000
50%    1.569434e+07   37.000000     70000.000000    0.000000
75%    1.575036e+07   46.000000     88000.000000    1.000000
max    1.581524e+07   60.000000    15

In [9]:
#Selección de variables
X = df.drop(["Purchased"], axis=1)
y = df["Purchased"]

#Preprocesamiento
scaler = StandardScaler()
X = scaler.fit_transform(X)

#División de datos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# --------------------
# MÉTODOS DE BALANCEO DE CLASES
# --------------------
# 1. Sobremuestreo aleatorio
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 3. Submuestreo aleatorio
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# 4. NearMiss (submuestreo basado en distancia)
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

# --------------------
# EVALUACIÓN DE MÉTODOS DE BALANCEO
# --------------------
resultados_balanceo = {}

def evaluar_balanceo(X_train_resampled, y_train_resampled, metodo):
    model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred) * 100
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) * 100
    resultados_balanceo[metodo] = (f1, auc)
    print(f"\nResultados con {metodo}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred) * 100:.2f}%")
    print(f"F1-Score: {f1:.2f}%")
    print(f"AUC-ROC: {auc:.2f}%")

evaluar_balanceo(X_train_ros, y_train_ros, "Sobremuestreo Aleatorio")
evaluar_balanceo(X_train_smote, y_train_smote, "SMOTE")
evaluar_balanceo(X_train_rus, y_train_rus, "Submuestreo Aleatorio")
evaluar_balanceo(X_train_nm, y_train_nm, "NearMiss")

# Seleccionar el mejor método de balanceo
mejor_metodo = max(resultados_balanceo, key=lambda k: resultados_balanceo[k])
print(f"\nEl mejor método de balanceo es: {mejor_metodo} con F1-Score: {resultados_balanceo[mejor_metodo][0]:.2f}% y AUC-ROC: {resultados_balanceo[mejor_metodo][1]:.2f}%")

# Obtener los datos balanceados del mejor método encontrado
if mejor_metodo == "Sobremuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_ros, y_train_ros
elif mejor_metodo == "SMOTE":
    X_train_resampled, y_train_resampled = X_train_smote, y_train_smote
elif mejor_metodo == "Submuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_rus, y_train_rus
elif mejor_metodo == "NearMiss":
    X_train_resampled, y_train_resampled = X_train_nm, y_train_nm

# --------------------
# MÉTODO 1: Ajuste manual de hiperparámetros
# --------------------
manual_model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)  # Ejemplo de hiperparámetros ajustados manualmente
manual_model.fit(X_train_resampled, y_train_resampled)  # Entrenar modelo

# Predicciones
y_pred_manual = manual_model.predict(X_test)

# Evaluación del modelo ajustado manualmente
print("\nResultados con ajuste manual de hiperparámetros:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, manual_model.predict_proba(X_test)[:,1]) * 100:.2f}%")

# --------------------
# MÉTODO 2: Ajuste automático con GridSearchCV
# --------------------
parametros = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],  # Diferentes valores de regularización
    "solver": ["liblinear", "lbfgs"]  # Diferentes algoritmos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=500), parametros, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)  # Entrenar búsqueda de hiperparámetros

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Evaluación del mejor modelo
print("\nResultados con ajuste automático de hiperparámetros:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor Accuracy en validación cruzada: {grid_search.best_score_ * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_best) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_best) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) * 100:.2f}%")


# --------------------
# ENTRENAMIENTO FINAL Y EVALUACIÓN
# --------------------

# Entrenar el mejor modelo con los datos balanceados
best_model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones finales con el mejor modelo y el mejor balanceo
y_pred_final = best_model.predict(X_test)

# Crear DataFrame de resultados
resultados = pd.DataFrame({
    "Real": y_test.values,
    "Predicción": y_pred_final
})

# Mapear valores 0 y 1 a etiquetas comprensibles
resultados["Real"] = resultados["Real"].map({0: "No clase y", 1: "clase y"})
resultados["Predicción"] = resultados["Predicción"].map({0: "No clase y", 1: "clase y"})

# Mostrar las primeras filas de la tabla
print(resultados.head(10))


Resultados con Sobremuestreo Aleatorio:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.05%

Resultados con SMOTE:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.12%

Resultados con Submuestreo Aleatorio:
Accuracy: 87.50%
Precision: 80.00%
Recall: 85.71%
F1-Score: 82.76%
AUC-ROC: 97.12%

Resultados con NearMiss:
Accuracy: 88.75%
Precision: 91.30%
Recall: 75.00%
F1-Score: 82.35%
AUC-ROC: 96.77%

El mejor método de balanceo es: SMOTE con F1-Score: 84.21% y AUC-ROC: 97.12%

Resultados con ajuste manual de hiperparámetros:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.12%

Resultados con ajuste automático de hiperparámetros:
Mejores parámetros: {'C': 1, 'solver': 'liblinear'}
Mejor Accuracy en validación cruzada: 84.88%
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.25%
         Real  Predicción
0  No clase y  No clase y
1     clase y     clase y
2  No clase y  N

Iris

In [11]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df["target"] = data.target 
df.info()
print(df["target"].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [12]:
#Selección de variables
X = df.drop(["target"], axis=1)
y = df["target"]

#Preprocesamiento
scaler = StandardScaler()
X = scaler.fit_transform(X)

#División de datos 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# --------------------
# MÉTODOS DE BALANCEO DE CLASES
# --------------------
# 1. Sobremuestreo aleatorio
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 3. Submuestreo aleatorio
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# 4. NearMiss (submuestreo basado en distancia)
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

# --------------------
# EVALUACIÓN DE MÉTODOS DE BALANCEO
# --------------------
resultados_balanceo = {}

def evaluar_balanceo(X_train_resampled, y_train_resampled, metodo):
    model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred) * 100
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) * 100
    resultados_balanceo[metodo] = (f1, auc)
    print(f"\nResultados con {metodo}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred) * 100:.2f}%")
    print(f"F1-Score: {f1:.2f}%")
    print(f"AUC-ROC: {auc:.2f}%")

evaluar_balanceo(X_train_ros, y_train_ros, "Sobremuestreo Aleatorio")
evaluar_balanceo(X_train_smote, y_train_smote, "SMOTE")
evaluar_balanceo(X_train_rus, y_train_rus, "Submuestreo Aleatorio")
evaluar_balanceo(X_train_nm, y_train_nm, "NearMiss")

# Seleccionar el mejor método de balanceo
mejor_metodo = max(resultados_balanceo, key=lambda k: resultados_balanceo[k])
print(f"\nEl mejor método de balanceo es: {mejor_metodo} con F1-Score: {resultados_balanceo[mejor_metodo][0]:.2f}% y AUC-ROC: {resultados_balanceo[mejor_metodo][1]:.2f}%")

# Obtener los datos balanceados del mejor método encontrado
if mejor_metodo == "Sobremuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_ros, y_train_ros
elif mejor_metodo == "SMOTE":
    X_train_resampled, y_train_resampled = X_train_smote, y_train_smote
elif mejor_metodo == "Submuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_rus, y_train_rus
elif mejor_metodo == "NearMiss":
    X_train_resampled, y_train_resampled = X_train_nm, y_train_nm

# --------------------
# MÉTODO 1: Ajuste manual de hiperparámetros
# --------------------
manual_model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)  # Ejemplo de hiperparámetros ajustados manualmente
manual_model.fit(X_train_resampled, y_train_resampled)  # Entrenar modelo

# Predicciones
y_pred_manual = manual_model.predict(X_test)

# Evaluación del modelo ajustado manualmente
print("\nResultados con ajuste manual de hiperparámetros:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, manual_model.predict_proba(X_test)[:,1]) * 100:.2f}%")

# --------------------
# MÉTODO 2: Ajuste automático con GridSearchCV
# --------------------
parametros = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],  # Diferentes valores de regularización
    "solver": ["liblinear", "lbfgs"]  # Diferentes algoritmos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=500), parametros, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)  # Entrenar búsqueda de hiperparámetros

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Evaluación del mejor modelo
print("\nResultados con ajuste automático de hiperparámetros:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor Accuracy en validación cruzada: {grid_search.best_score_ * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_best) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_best) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) * 100:.2f}%")


# --------------------
# ENTRENAMIENTO FINAL Y EVALUACIÓN
# --------------------

# Entrenar el mejor modelo con los datos balanceados
best_model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones finales con el mejor modelo y el mejor balanceo
y_pred_final = best_model.predict(X_test)

# Crear DataFrame de resultados
resultados = pd.DataFrame({
    "Real": y_test.values,
    "Predicción": y_pred_final
})

# Mapear valores 0 y 1 a etiquetas comprensibles
resultados["Real"] = resultados["Real"].map({0: "No clase y", 1: "clase y"})
resultados["Predicción"] = resultados["Predicción"].map({0: "No clase y", 1: "clase y"})

# Mostrar las primeras filas de la tabla
print(resultados.head(10))


Resultados con Sobremuestreo Aleatorio:
Accuracy: 98.25%
Precision: 98.59%
Recall: 98.59%
F1-Score: 98.59%
AUC-ROC: 99.80%

Resultados con SMOTE:
Accuracy: 98.25%
Precision: 98.59%
Recall: 98.59%
F1-Score: 98.59%
AUC-ROC: 99.84%

Resultados con Submuestreo Aleatorio:
Accuracy: 97.37%
Precision: 98.57%
Recall: 97.18%
F1-Score: 97.87%
AUC-ROC: 99.84%

Resultados con NearMiss:
Accuracy: 96.49%
Precision: 97.18%
Recall: 97.18%
F1-Score: 97.18%
AUC-ROC: 99.80%

El mejor método de balanceo es: SMOTE con F1-Score: 98.59% y AUC-ROC: 99.84%

Resultados con ajuste manual de hiperparámetros:
Accuracy: 98.25%
Precision: 98.59%
Recall: 98.59%
F1-Score: 98.59%
AUC-ROC: 99.84%

Resultados con ajuste automático de hiperparámetros:
Mejores parámetros: {'C': 0.1, 'solver': 'lbfgs'}
Mejor Accuracy en validación cruzada: 98.08%
Accuracy: 98.25%
Precision: 98.59%
Recall: 98.59%
F1-Score: 98.59%
AUC-ROC: 99.84%
         Real  Predicción
0     clase y     clase y
1  No clase y  No clase y
2  No clase y  No 

In [ ]:
# --------------------
# PRUEBA Y EJECUCIÓN DEL MODELO
# --------------------


# Buscamos un registro de un paciente con cancer para pasar los mismos datos y probar que el modelo predice correctamente
print(df[df["target"] == 1].index) # Seleccionamos la 37
print(df.loc[37])

import numpy as np

# Mapeo de características a nombres en castellano (opcional)
nombres_caracteristicas = {
    'mean radius': 'Radio promedio',
    'mean texture': 'Textura promedio',
    'mean perimeter': 'Perímetro promedio',
    'mean area': 'Área promedio',
    'mean smoothness': 'Suavidad promedio',
    'mean compactness': 'Compacidad promedio',
    'mean concavity': 'Concavidad promedio',
    'mean concave points': 'Puntos cóncavos promedio',
    'mean symmetry': 'Simetría promedio',
    'mean fractal dimension': 'Dimensión fractal promedio',
    'radius error': 'Error de radio',
    'texture error': 'Error de textura',
    'perimeter error': 'Error de perímetro',
    'area error': 'Error de área',
    'smoothness error': 'Error de suavidad',
    'compactness error': 'Error de compacidad',
    'concavity error': 'Error de concavidad',
    'concave points error': 'Error de puntos cóncavos',
    'symmetry error': 'Error de simetría',
    'fractal dimension error': 'Error de dimensión fractal',
    'worst radius': 'Radio máximo',
    'worst texture': 'Textura máxima',
    'worst perimeter': 'Perímetro máximo',
    'worst area': 'Área máxima',
    'worst smoothness': 'Suavidad máxima',
    'worst compactness': 'Compacidad máxima',
    'worst concavity': 'Concavidad máxima',
    'worst concave points': 'Puntos cóncavos máximos',
    'worst symmetry': 'Simetría máxima',
    'worst fractal dimension': 'Dimensión fractal máxima'
}

# Creamos la función para ejecutar el modelo con nuevos datos
def solicitar_datos_y_predecir(modelo, feature_names, scaler):
    print("\nIngrese los valores para realizar una predicción:")
    valores = []
    
    for feature in feature_names:
        nombre_caracteristica = nombres_caracteristicas.get(feature, feature)  
        while True:
            try:
                valor = float(input(f"{nombre_caracteristica}: "))
                valores.append(valor)
                break
            except ValueError:
                print("Entrada inválida. Ingrese un número válido.")
    
    # Convertir la entrada en un array de numpy y escalarlo con el scaler ya entrenado
    datos_nuevos = np.array(valores).reshape(1, -1)
    datos_nuevos_escalados = scaler.transform(datos_nuevos)
    
    # Realizar predicción
    prediccion = modelo.predict(datos_nuevos_escalados)[0]
    probabilidad = modelo.predict_proba(datos_nuevos_escalados)[0][prediccion] * 100
    
    # Mostrar resultado
    resultado = "Maligno" if prediccion == 1 else "Benigno"
    print(f"\nPredicción: {resultado} ({probabilidad:.2f}% de confianza)\n")

# Llamar a la función con el scaler ya ajustado
solicitar_datos_y_predecir(best_model, data.feature_names, scaler)


Index([ 19,  20,  21,  37,  46,  48,  49,  50,  51,  52,
       ...
       553, 554, 555, 556, 557, 558, 559, 560, 561, 568],
      dtype='int64', length=357)
mean radius                 13.030000
mean texture                18.420000
mean perimeter              82.610000
mean area                  523.800000
mean smoothness              0.089830
mean compactness             0.037660
mean concavity               0.025620
mean concave points          0.029230
mean symmetry                0.146700
mean fractal dimension       0.058630
radius error                 0.183900
texture error                2.342000
perimeter error              1.170000
area error                  14.160000
smoothness error             0.004352
compactness error            0.004899
concavity error              0.013430
concave points error         0.011640
symmetry error               0.026710
fractal dimension error      0.001777
worst radius                13.300000
worst texture               22.810000
worst

c:\Users\lalos\OneDrive\Escritorio\virtual2\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


: 